In [2]:
# Imports

from gedcom.element.individual import IndividualElement
from gedcom.element.element import Element
from gedcom.parser import Parser

# Théoph
# from ipynb.fs.full.explorertheoph import *

# Alice
# from ipynb.fs.full.explorer import *

# Alison
# from exploreralison import *

Information about Elizabeth II Alexandra Mary Windsor:
Agnes de Paganel
F


In [3]:
file_path = 'Queen_Eliz_II.ged'
gedcom_parser = Parser()
gedcom_parser.parse_file(file_path)
root_child_elements = gedcom_parser.get_root_child_elements()

In [27]:
def find_IndividualElement(first_names,last_name):
    """
    Get Individual element of ged with first names and last name

    Parameters
    ---
    first_names : str
        complete first names of the individual
    last_name : str
        last names of the individual 

    Returns 
    ---
    bool 
        Tells if the individual was found
    IndividualElement
        IndividualElement corresponding to the individual in the ged

    """
    for element in root_child_elements:
        if isinstance(element, IndividualElement):
            if element.surname_match(last_name):
                return (True, element)
    return [False, element]

In [35]:
def get_name_str(individual) :
    (first, last) = individual.get_name()
    name = first + " " + last
    return name

def print_name(individual) :
    print(get_name_str(individual))

Voici une série de tests pour prendre en main les principales fonctions de gedcom, qui permettent de trouver les familles, enfants, parents, époux d'un individu. 

In [93]:
individual = find_IndividualElement("De Brus Robert Bruce", "de Brusse")[1]

# Trouve la famille dont l'individu est un des parents
# sous forme d'une liste à un ou plusieurs éléments
gedcom_parser.get_families(individual)

# Trouve la famille dont l'individu est enfant
# sous forme d'une liste à un ou plusieurs éléments
gedcom_parser.get_families(individual, "FAMC")

# Nombre de familles par individu 
liste_individus = gedcom_parser.get_root_child_elements()
list_spouse = [0,0,0,0]
list_child = [0,0,0,0]
for x in liste_individus :
    if x.get_tag() == "INDI" :
        nb_spouse = len(gedcom_parser.get_families(x))
        nb_child = len(gedcom_parser.get_families(x,"FAMC"))
        if nb_spouse < 4 :
            list_spouse[nb_spouse] += 1
        if nb_child < 4 :
            if nb_child == 0 :
                print_name(x)
            list_child[nb_child] += 1
print("Nombre d'individus marriés : ", list_spouse)
print("Nombre d'individus enfants d'une famille : ", list_child)

Blichilde Countess Of Maine 
N.N. de Chartres 
Ostrida Regenwaldsdatter [Countess Of Orn] Ostrida Regenwaldsdatter Regenwaldsdóttir, Orkney
Grelod Gruaidh Dungadsdotter
Frille II til Sigurd (mistress), I 
Grelod Gruaidh Dungadsdotter
"Countess" Mrs. (Einar) (Rognvaldsson) 
NN (Frille) til Ragnvald Mørejarl (på Møre), (mistress) 
Gundella? Eysteinsdatter 
 
 
NN Wife of Heytir Gorrson 
Gorr Thorrasson, Alfheim 
Ikutiera (Princess in FINLAND) 
Caroline of Nassau- Usingen 
Richende (Richilde) de Bourges 
Eudes II, Comte de Chartres 
Robert I (III), Count of Auvergne 
Ermgard de Provence, 
Ermengarde Duchesse DAnjou deBretagne 
Princess Gerberge De LORRAINE 
OREGUEN 
Hildebrante of France 
Beatrice de Morvois 
Cunegonde 
Hildegarde of Swabia 
Clothilde (Doda) of Saxony 
Oda of Swabia1 
Arthemia of Geneva 
Sedeleuba Chroma de Burgundy
Theodelinde 
Beatrice de Teyden 
Isabella of Mar 
Anabella Drummond 
Guillaume Paynel, Seigneur de Hambye & Moutiers-Hubert 
Lesline de Fontenay 
Richard de S

Tous les individus sont mariés au moins une fois (étonnant ?). Ce qui est réellement étonnant par contre, c'est que certains individus n'ont pas de famille dont ils sont enfants. 

C'est d'autant plus étrange que lorsque je prends un individu qui n'a a priori pas de FAMC et que je cherche sa FAMC, je la trouve, mais l'individu ne figure pas dans sa propre famille... 

In [91]:
strange = find_IndividualElement("Blichilde Countess Of Maine","")[1]
gedcom_parser.get_families(strange,"FAMC")
for fam in gedcom_parser.get_family_members(gedcom_parser.get_families(strange,"FAMC")[0]) :
    print_name(fam)

len(gedcom_parser.get_families(strange,"FAMC"))



William VIII Gascoigne
Matilda de Gawkethorp
SIR WILLIAM VII KNIGHT ENGLAND Gascoigne


1

In [26]:
# Trouve les parents d'un individu
parents = gedcom_parser.get_parents(individual)
print("Parents : ", get_name_str(parents[0]), " and ", get_name_str(parents[1]))

# Pas de fonction pour trouver les enfants d'un individu

# Trouve les membres de la famille de l'individu (père, mère, enfants)
family = gedcom_parser.get_families(individual)[0]
members = gedcom_parser.get_family_members(family)

print("Membres de la famille : ")
for x in members :
    print(get_name_str(x))

# On vérifie qu'il s'agit bien des enfants de l'individu
# en prenant l'un des membres de la famille
enfant = find_IndividualElement("(Baron) Adam", "de Brus")[1]
parents = gedcom_parser.get_parents(enfant)
print("Parents de Adam de Brus : ", get_name_str(parents[0]), " and ", get_name_str(parents[1]))

Parents :  Ragnvald (Rognvald) II Brusesson [Earl Of Orkney]   and  Arlogia Countess Of Orkney 
Membres de la famille : 
De Brus Robert Bruce de Brusse
Emma De Brittany or DE BRETAGNE
(Baron) Adam de Brus
(NN) de BRAOSE De Braose
Parents de Adam de Brus :  De Brus Robert Bruce de Brusse  and  Emma De Brittany or DE BRETAGNE


In [33]:
# On en déduit une manière simple de trouver les enfants et le conjoint d'un individu.

def get_spouse(individual) :
    """
    Returns spouse(s) of individual

    Parameters
    ---
    individual : IndividualElement

    Returns 
    ---
    list of IndividualElement

    """
    family = gedcom_parser.get_families(individual)
    spouse = []
    if individual.get_gender() == "M" :
        for fam in family :
            spouse.append(gedcom_parser.get_family_members(fam)[1])
    else :
        for fam in family :
            spouse.append(gedcom_parser.get_family_members(fam)[0])
    return spouse
    

In [37]:
# Test ok

# Avec le mari
spouse = get_spouse(individual)
for x in spouse :
    print_name(x)

# Avec l'épouse 
get_spouse(get_spouse(individual)[0])[0] == individual


Emma De Brittany or DE BRETAGNE


True

In [40]:
def get_children(individual) :
    """
    Returns spouse(s) of individual

    Parameters
    ---
    individual : IndividualElement

    Returns 
    ---
    list of IndividualElement
    """

    family = gedcom_parser.get_families(individual)
    children = []
    for fam in family :
        members = gedcom_parser.get_family_members(fam)
        for i in range (2, len(members)) :
            children.append(members[i])
    return children


In [42]:
# Test ok

children = get_children(individual)
for child in children :
    print_name(child)


(Baron) Adam de Brus
(NN) de BRAOSE De Braose


In [68]:
def get_siblings(individual) :
    """
    Returns siblings of individual

    Parameters
    ---
    individual : IndividualElement

    Returns 
    ---
    list of IndividualElement
    """

    family = gedcom_parser.get_families(individual,"gedcom_parser.tags.GEDCOM_TAG_FAMILY_CHILD")[0]
    members = gedcom_parser.get_family_members(family)
    siblings = []
    for i in range (2, len(members)) :
        if members[i] != individual :
            siblings.append(members[i])
    return siblings
    


In [78]:
# Test

individual = find_IndividualElement("(Baron) Adam", "de Brus")[1]
family = gedcom_parser.get_families(individual,"gedcom_parser.tags.GEDCOM_TAG_FAMILY_CHILD")
family
gedcom_parser.get_families(,"gedcom_parser.tags.GEDCOM_TAG_FAMILY_CHILD")

[]

Tentons maintenant une recherche de parenté entre deux individus. Nous allons commencer par coder des fonctions de tests simples, pour déterminer si deux individus sont liés ou non.

In [43]:
def is_family(x, y) :
    """
    Indicates if two individuals are in the same close family

    Parameters
    ---
    x : IndividualElement
    y : IndividualElement

    Returns 
    ---
    bool
        False if x and y are not in the same close family
    FamilyElement
        the family to which x and y belong if any
    """

    fam_x = gedcom_parser.get_families(x)
    fam_y = gedcom_parser.get_families(y)

    for fam in fam_x :
        if fam in fam_y :
            return fam
    return False
    

In [60]:
# Test

# Pour un couple d'individus mariés
x = individual
y = get_spouse(individual)[0]
print(is_family(x,y))

# Pour des individus quelconques
x = individual
y = find_IndividualElement("SIR WILLIAM VII KNIGHT ENGLAND", "Gascoigne")[1]
print(is_family(x,y))

0 @F77@ FAM

False
